In [9]:
#pip install pyvo mocpy ipyaladin

In [10]:
# Access astronomical databases
from pyvo import registry  # version >=1.4.1 

# Moc and HEALPix tools
from mocpy import MOC

# Sky visualization
from ipyaladin import Aladin    # version >=0.3.0

# For plots
import matplotlib.pyplot as plt

from astropy.io import fits

# Welcome to VizieR example workflow

[![Vizier](https://custom-icon-badges.demolab.com/badge/Vizier-gray.svg?logo=vizier&logoColor=orange&logoWidth=20)](https://vizier.cds.unistra.fr/viz-bin/VizieR "https://vizier.cds.unistra.fr/viz-bin/VizieR")

It is a generic notebook, highlighting what can be done once you chose a catalog. This workflow is suggested by [CDS](https://cdsweb.unistra.fr/) (Strasbourg Astronomical Data Center, house of [VizieR](https://vizier.cds.unistra.fr/viz-bin/VizieR)).

--------------------------------------------------------

Modify the <a href='https://vizier.cds.unistra.fr/'>VizieR catalogue</a> name (variable <b>CATALOGUE</b>) and anything else you might feel like exploring!

## 1. Setup

This example notebook has the following dependencies: 

**Required**
- pyvo : this library facilitates the access to the Virtual Observatory (VO) resources. VizieR is part of the VO.
This notebook needs version >=1.4.1

**Optional, for visualization**
- ipyaladin : this is the Aladin-lite sky viewer, bundled as a jupyter widget. It allows to plot catalogs and multi-order coverages (MOC)
- matplotlib : an other option to see catalog points and MOCs

## 2. Metadata exploration with the Virtual Observatory registry

This part uses [pyvo](https://pyvo.readthedocs.io/en) to connect to the VO registry.

In [11]:
# the catalogue name in VizieR
CATALOGUE = "VIII/28"

We first retrieve catalogue information.

In [12]:
# each resource in the VO has an identifier, called ivoid. For vizier catalogs,
# the VO ids can be constructed like this:
catalogue_ivoid = f"ivo://CDS.VizieR/{CATALOGUE}"
# the actual query to the registry
voresource = registry.search(ivoid=catalogue_ivoid)[0]

In [13]:
# We can print metadata information about the catalogue
voresource.describe(verbose=True)

Bell Laboratories H I Survey
Short Name: VIII/28
IVOA Identifier: ivo://cds.vizier/viii/28
Access modes: tap#aux, web
Multi-capability service -- use get_service()

The sky north of declination -40{deg} was observed in the 21cm line of atomic
hydrogen with the FWHM=2{deg} beam of the 20 foot horn reflector at AT&T Bell
Laboratories, Crawford Hill. The survey covers a velocity range of 654km/s
centered on the Galactic standard of rest, with 5.3km/s wide filters. This
survey is distinguished by its sensitivity to low surface brightness features
(antenna temperature about 50mK) and relative freedom from sidelobe
contamination. High-velocity clouds are extracted and catalogued
automatically. The data are presented in declination zones in equatorial and
polar coordinates, and as R.A. - velocity images. Introduction: Observations
were made as drift scans along even declinations between -40{deg} and +90{deg}
at the epoch (1981) of the observations. The horn antenna has a FWHM beam size
of 2{d

We can also inspect in details the `resource` object and access the attributes not provided by the describe method. See for example, the first author of a resource: 

In [14]:
voresource.creators[0]

'Stark A.A.'

## 3. Access the tabular data of this catalog

We can have a look at the tables available in the catalogue.

In [15]:
tables = voresource.get_tables()
tables

{'VIII/28/ravel': <VODataServiceTable name="VIII/28/ravel">... 3 columns ...</VODataServiceTable>,
 'VIII/28/sky': <VODataServiceTable name="VIII/28/sky">... 5 columns ...</VODataServiceTable>}

In [16]:
# We can also extract the tables names for later use
tables_names = list(tables.keys())
tables_names

['VIII/28/ravel', 'VIII/28/sky']

The actual data can then be access using any of the ``access_modes`` of the voresource.

In [17]:
voresource.access_modes()

{'tap#aux', 'web'}

The web access is found by following the ``reference_url``

In [18]:
voresource.reference_url

'https://cdsarc.cds.unistra.fr/viz-bin/cat/VIII/28'

### 3.1 Execute a SQL/ADQL query

The ``tap#aux`` in the ``access_mode`` response indicates that we can also do a SQL/ADQL query for these VizieR tables.

On the first table of the catalogue, we execute an <a href='https://www.ivoa.net/documents/latest/ADQL.html'>ADQL</a> query.

In [19]:
# get the first table of the catalogue
first_table_name = tables_names[1]

# execute a synchronous ADQL query
tap_service = voresource.get_service("tap")
tap_records = voresource.get_service("tap").run_sync(
    f'select TOP 10 * from "{first_table_name}"',
)
tap_records

<DALResultsTable length=10>
recno  cVel   FITS-Nfile   FITS-Wfile   FITS-Sfile 
      km / s                                       
int32 int16     object       object       object   
----- ------ ------------ ------------ ------------
    1   -360 npcap001.fit spcap001.fit whsky001.fit
    2   -355 npcap002.fit spcap002.fit whsky002.fit
    3   -350 npcap003.fit spcap003.fit whsky003.fit
    4   -345 npcap004.fit spcap004.fit whsky004.fit
    5   -340 npcap005.fit spcap005.fit whsky005.fit
    6   -335 npcap006.fit spcap006.fit whsky006.fit
    7   -330 npcap007.fit spcap007.fit whsky007.fit
    8   -325 npcap008.fit spcap008.fit whsky008.fit
    9   -320 npcap009.fit spcap009.fit whsky009.fit
   10   -315 npcap010.fit spcap010.fit whsky010.fit

In [20]:
##! unzip ~/LBASS/VIII_28.tar.zip -d ~/LBASS

In [22]:
hdul = fits.open("VIII_28.tar/fits/whsky001.fit")
hdul.info()
data = hdul[0].data
print(data)
hdul.close()

Filename: VIII_28.tar/fits/whsky001.fit
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      31   (800, 400, 1)   int16 (rescales to float32)   
[[[-0.08216609 -0.08216609 -0.08216609 ... -0.08216609 -0.08216609
   -0.08216609]
  [-0.079271   -0.079271   -0.07929245 ... -0.07922812 -0.07924956
   -0.079271  ]
  [-0.075561   -0.0756039  -0.07564679 ... -0.07150789 -0.07155077
   -0.07161511]
  ...
  [        nan         nan         nan ...         nan         nan
           nan]
  [        nan         nan         nan ...         nan         nan
           nan]
  [        nan         nan         nan ...         nan         nan
           nan]]]


In [24]:
whsky001 = fits.open("VIII_28.tar/fits/whsky001.fit")
npcap001 = fits.open("VIII_28.tar/fits/npcap001.fit")
spcap001 = fits.open("VIII_28.tar/fits/spcap001.fit")
print(whsky001[0].header)

SIMPLE  =                    T /                                                BITPIX  =                   16 /                                                NAXIS   =                    3 /                                                NAXIS1  =                  800 /                                                NAXIS2  =                  400 /                                                NAXIS3  =                    1 /                                                CTYPE1  = 'GLON    '           /                                                CRVAL1  =           0.00000000 /                                                CDELT1  =         -0.450563204 /                                                CRPIX1  =           800.000000 /                                                CROTA1  =           0.00000000 /                                                CTYPE2  = 'GLAT    '           /                                                CRVAL2  =           0.00000000 /        

In [25]:
print(npcap001[0].header)

SIMPLE  =                    T /                                                BITPIX  =                   16 /                                                NAXIS   =                    3 /                                                NAXIS1  =                  400 /                                                NAXIS2  =                  400 /                                                NAXIS3  =                    1 /                                                CTYPE1  = 'GLON-SIN'           /                                                CRVAL1  =           0.00000000 /                                                CDELT1  =         -0.300751880 /                                                CRPIX1  =           200.500000 /                                                CROTA1  =           0.00000000 /                                                CTYPE2  = 'GLAT-SIN'           /                                                CRVAL2  =           90.0000000 /        

In [26]:
print(spcap001[0].header)

SIMPLE  =                    T /                                                BITPIX  =                   16 /                                                NAXIS   =                    3 /                                                NAXIS1  =                  400 /                                                NAXIS2  =                  400 /                                                NAXIS3  =                    1 /                                                CTYPE1  = 'DX---SIN'           /                                                CRVAL1  =           0.00000000 /                                                CDELT1  =         -0.300751880 /                                                CRPIX1  =           200.500000 /                                                CROTA1  =           0.00000000 /                                                CTYPE2  = 'DY---SIN'           /                                                CRVAL2  =           0.00000000 /        

In [28]:
ravel001 = fits.open("VIII_28.tar/fits/ravel001.fit")
print(ravel001[0].header)

SIMPLE  =                    T /                                                BITPIX  =                   16 /                                                NAXIS   =                    3 /                                                NAXIS1  =                  361 /                                                NAXIS2  =                  228 /                                                NAXIS3  =                    1 /                                                CTYPE1  = 'RA      '           /                                                CRVAL1  =           0.00000000 /                                                CDELT1  =           1.00000000 /                                                CRPIX1  =           181.000000 /                                                CROTA1  =           0.00000000 /                                                CTYPE2  = 'FELO-LSR'           /                                                CRVAL2  =          -601524.110 /        